<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.4.1


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

  0%|          | 0/2 [00:00<?, ? splits/s]

Shuffling and writing examples to tensorflow_datasets/horses_or_humans/3.0.0.incompleteFF1QA9/horses_or_humans-train.tfrecord


Shuffling and writing examples to tensorflow_datasets/horses_or_humans/3.0.0.incompleteFF1QA9/horses_or_humans-test.tfrecord
Dataset horses_or_humans downloaded and prepared to ./tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 8s 297ms/step - loss: 0.6846 - accuracy: 0.5316 - val_loss: 0.6718 - val_accuracy: 0.5317
Epoch 2/10
26/26 [==============================] - 7s 256ms/step - loss: 0.6528 - accuracy: 0.6419 - val_loss: 0.6320 - val_accuracy: 0.6293
Epoch 3/10
26/26 [==============================] - 6s 240ms/step - loss: 0.6255 - accuracy: 0.6386 - val_loss: 0.5852 - val_accuracy: 0.8244
Epoch 4/10
26/26 [==============================] - 6s 240ms/step - loss: 0.5944 - accuracy: 0.6885 - val_loss: 0.5799 - val_accuracy: 0.6488
Epoch 5/10
26/26 [==============================] - 6s 238ms/step - loss: 0.5942 - accuracy: 0.6753 - val_loss: 0.4908 - val_accuracy: 0.7805
Epoch 6/10
26/26 [==============================] - 6s 231ms/step - loss: 0.5098 - accuracy: 0.7651 - val_loss: 0.5168 - val_accuracy: 0.6829
Epoch 7/10
26/26 [==============================] - 6s 236ms/step - loss: 0.4795 - accuracy: 0.7939 - val_loss: 0.4570 - val_accuracy: 0.7902
Epoch 

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5
26/26 - 7s - loss: 0.6642 - accuracy: 0.6083 - val_loss: 0.6568 - val_accuracy: 0.5463

Epoch 00001: saving model to weights.01-0.66.h5
Epoch 2/5
26/26 - 6s - loss: 0.6133 - accuracy: 0.6788 - val_loss: 0.7107 - val_accuracy: 0.4976

Epoch 00002: saving model to weights.02-0.71.h5
Epoch 3/5
26/26 - 6s - loss: 0.5714 - accuracy: 0.7153 - val_loss: 0.5199 - val_accuracy: 0.7902

Epoch 00003: saving model to weights.03-0.52.h5
Epoch 4/5
26/26 - 6s - loss: 0.4956 - accuracy: 0.7786 - val_loss: 0.4535 - val_accuracy: 0.8634

Epoch 00004: saving model to weights.04-0.45.h5
Epoch 5/5
26/26 - 6s - loss: 0.4474 - accuracy: 0.8114 - val_loss: 0.3889 - val_accuracy: 0.8927

Epoch 00005: saving model to weights.05-0.39.h5


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])

26/26 - 6s - loss: 0.6725 - accuracy: 0.5900 - val_loss: 0.6569 - val_accuracy: 0.8195

Epoch 00001: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2
26/26 - 7s - loss: 0.6484 - accuracy: 0.6399 - val_loss: 0.6028 - val_accuracy: 0.7366

Epoch 00001: saving model to model.h5
Epoch 2/2
26/26 - 6s - loss: 0.5678 - accuracy: 0.7397 - val_loss: 0.5183 - val_accuracy: 0.7610

Epoch 00002: saving model to model.h5


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 6s - loss: 0.6786 - accuracy: 0.5766 - val_loss: 0.6766 - val_accuracy: 0.5073
Epoch 2/50
26/26 - 6s - loss: 0.6321 - accuracy: 0.6399 - val_loss: 0.6196 - val_accuracy: 0.6780
Epoch 3/50
26/26 - 6s - loss: 0.5784 - accuracy: 0.7251 - val_loss: 0.5644 - val_accuracy: 0.6439
Epoch 4/50
26/26 - 6s - loss: 0.5380 - accuracy: 0.7421 - val_loss: 0.4940 - val_accuracy: 0.7610
Epoch 5/50
26/26 - 6s - loss: 0.4910 - accuracy: 0.7810 - val_loss: 0.4405 - val_accuracy: 0.7805
Epoch 6/50
26/26 - 6s - loss: 0.4400 - accuracy: 0.7981 - val_loss: 0.3993 - val_accuracy: 0.8488
Epoch 7/50
26/26 - 6s - loss: 0.3757 - accuracy: 0.8504 - val_loss: 0.3353 - val_accuracy: 0.8976
Epoch 8/50
26/26 - 6s - loss: 0.3376 - accuracy: 0.8540 - val_loss: 0.2772 - val_accuracy: 0.9317
Epoch 9/50
26/26 - 6s - loss: 0.2818 - accuracy: 0.8856 - val_loss: 0.2626 - val_accuracy: 0.9073
Epoch 10/50
26/26 - 6s - loss: 0.2220 - accuracy: 0.9392 - val_loss: 0.4112 - val_accuracy: 0.7854
Epoch 11/50
26/26 -

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 7s 236ms/step - loss: 0.6816 - accuracy: 0.5764 - val_loss: 0.6821 - val_accuracy: 0.4488
Epoch 2/5
26/26 [==============================] - 6s 228ms/step - loss: 0.6564 - accuracy: 0.6281 - val_loss: 0.6187 - val_accuracy: 0.7220
Epoch 3/5
26/26 [==============================] - 6s 228ms/step - loss: 0.6122 - accuracy: 0.7179 - val_loss: 0.5993 - val_accuracy: 0.6634
Epoch 4/5
26/26 [==============================] - 6s 228ms/step - loss: 0.5790 - accuracy: 0.7156 - val_loss: 0.5666 - val_accuracy: 0.6927
Epoch 5/5
26/26 [==============================] - 6s 227ms/step - loss: 0.5384 - accuracy: 0.7633 - val_loss: 0.5786 - val_accuracy: 0.6049


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.602190,0.674886,0.448780,0.682127
1,1,0.658151,0.644578,0.721951,0.618738
2,2,0.700730,0.609689,0.663415,0.599324
3,3,0.725061,0.573065,0.692683,0.566608
4,4,0.772506,0.523444,0.604878,0.578631


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])

Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
26/26 [==============================] - 7s 258ms/step - loss: 0.6954 - accuracy: 0.5237 - val_loss: 0.7205 - val_accuracy: 0.4341
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
26/26 [==============================] - 6s 237ms/step - loss: 0.6656 - accuracy: 0.5768 - val_loss: 0.6634 - val_accuracy: 0.5756
Epoch 3/5

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
26/26 [==============================] - 6s 227ms/step - loss: 0.6450 - accuracy: 0.6396 - val_loss: 0.6604 - val_accuracy: 0.5659
Epoch 4/5

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
26/26 [==============================] - 6s 225ms/step - loss: 0.6361 - accuracy: 0.6680 - val_loss: 0.6564 - val_accuracy: 0.5854
Epoch 5/5

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
26/26 [==============================] - 6s 226ms/step - loss: 0.6348 -

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 7s 264ms/step - loss: 0.6946 - accuracy: 0.5275 - val_loss: 0.6262 - val_accuracy: 0.6683
Epoch 2/50
26/26 [==============================] - 6s 237ms/step - loss: 0.6221 - accuracy: 0.6584 - val_loss: 0.5814 - val_accuracy: 0.7659
Epoch 3/50
26/26 [==============================] - 6s 238ms/step - loss: 0.5651 - accuracy: 0.7326 - val_loss: 0.5850 - val_accuracy: 0.6537

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
Epoch 4/50
26/26 [==============================] - 6s 235ms/step - loss: 0.5077 - accuracy: 0.7973 - val_loss: 0.4840 - val_accuracy: 0.8000
Epoch 5/50
26/26 [==============================] - 6s 241ms/step - loss: 0.4674 - accuracy: 0.8258 - val_loss: 0.4769 - val_accuracy: 0.8098
Epoch 6/50
26/26 [==============================] - 6s 236ms/step - loss: 0.4540 - accuracy: 0.8119 - val_loss: 0.4503 - val_accuracy: 0.8146
Epoch 7/50
26/26 [==============================] - 6s 233ms/step -

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir log_dir